**Dataset:**

I've downloaded a large subset of the item-review data json files. The first thing to do, is to get rid of all the extra attributes.

I just want all the item descriptions to pretrain fast text on

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
%cd gdrive
%cd MyDrive/
%cd "RetailGo RecSys"
%cd "Experiments (Amazon Products)"
%cd "Datasets"

/content/gdrive
/content/gdrive/MyDrive
/content/gdrive/MyDrive/RetailGo RecSys
/content/gdrive/MyDrive/RetailGo RecSys/Experiments (Amazon Products)
/content/gdrive/MyDrive/RetailGo RecSys/Experiments (Amazon Products)/Datasets


In [4]:
!pip install tqdm --q
!pip install nltk

In [5]:
import os
import json
from tqdm import tqdm
import re

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
!pip install gensim --q

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stopwords = stopwords.words("english")

In [8]:
def preprocess(description):

  if type(description) == list:
    description = ' '.join(description)

  # Drop all non-letter characters
  description = re.sub("[^a-zA-Z ]", "", description)

  # Split by space
  description = description.split()

  # Remove stopwords.
  return [word for word in description if word not in stopwords and len(word) > 2]

preprocess("Some text that has, 32 ounces -- per case numbers: 10gou sohrguohsrguh")

['Some', 'text', 'ounces', 'per', 'case', 'numbers', 'gou', 'sohrguohsrguh']

In [40]:
product_descriptions = []

number_of_products_per_category = 2000

for category in tqdm(os.listdir()):

  if ".json" not in category:
    continue

  number_of_products_in_category = 0

  # For each category, iterate through each item.
  with open(category, "r+") as products:
    for product_line in products.readlines():
        if number_of_products_in_category > number_of_products_per_category:
          break
        product = json.loads(product_line)
        try:
          if "description" not in product:
            continue

          product_words = preprocess(product["description"])

          if len(product_words) == 0:
            continue

          product_descriptions.append(product_words)
          number_of_products_in_category += 1

        except:
          assert False

  print("\nFor category:", category, "Sample Product Description:", product_descriptions[-1])

  9%|▉         | 1/11 [00:03<00:33,  3.36s/it]


For category: meta_Grocery_and_Gourmet_Food.json Sample Product Description: ['Net', 'Carbs', 'Total', 'Carbs', 'NonImpact', 'Carbs', 'gThe', 'nonimpact', 'carbs', 'derived', 'Maltitol', 'sugar', 'alcohol', 'dietary', 'fiber', 'Milk', 'Chocolate', 'lovers', 'dream', 'Includes', 'delightful', 'assortment', 'Sugar', 'Added', 'Milk', 'Chocolate', 'bars', 'Maltitol', 'wheat', 'gluten', 'free', 'Cocoa', 'Butter', 'Cream', 'Milk', 'Cocoa', 'Mass', 'Soy', 'Lecithin', 'Vanilla', 'may', 'contain', 'traces', 'various', 'nuts', 'Statements', 'regarding', 'dietary', 'supplements', 'evaluated', 'FDA', 'intended', 'diagnose', 'treat', 'cure', 'prevent', 'disease', 'health', 'condition']


 18%|█▊        | 2/11 [00:18<01:32, 10.28s/it]


For category: meta_Sports_and_Outdoors.json Sample Product Description: ['These', 'good', 'general', 'purpose', 'glasses', 'They', 'provide', 'good', 'visual', 'acuity', 'excellent', 'glare', 'protection', 'Very', 'useful', 'Retinitis', 'Pigmentosa', 'diabetic', 'retinopathy', 'pre', 'cataract', 'glaucoma', 'These', 'glasses', 'provide', 'light', 'transmission', 'blue', 'light', 'infrared', 'transmissionFeatures', 'Color', 'Medium', 'Amber', 'Filters', 'Protection', 'SideShield', 'brow', 'protection', 'Universal', 'size', 'faces', 'framesFrame', 'Dimensions', 'Sizing', 'Inside', 'Outside', 'Temple', 'Eye', 'mmManufacturer', 'NoIROrigin', 'USAModel']


 27%|██▋       | 3/11 [00:19<00:48,  6.08s/it]


For category: meta_All_Beauty.json Sample Product Description: ['Kevin', 'Murphy', 'sticky', 'Business', 'Medium', 'hold']


 36%|███▋      | 4/11 [00:20<00:29,  4.21s/it]


For category: meta_AMAZON_FASHION.json Sample Product Description: ['Sparkle', 'Pin', 'Love', 'Jesus', 'bright', 'sparkley', 'colors', 'wonderful', 'wear', 'year', 'around']


100%|██████████| 11/11 [00:24<00:00,  2.25s/it]


For category: meta_Arts_Crafts_and_Sewing.json Sample Product Description: ['These', 'AllPurpose', 'shears', 'ideal', 'scrapbooking', 'papercrafts', 'part', 'Mundials', 'ergonomic', 'ambidextrous', 'CushionSoft', 'line', 'handles', 'enlarged', 'finger', 'bows', 'cushioned', 'inserts', 'soft', 'touch', 'The', 'stainless', 'steel', 'blades', 'feature', 'microserrated', 'edge', 'ensuring', 'clean', 'cut', 'even', 'sheerest', 'fabrics', 'Ideal', 'crafting', 'enthusiast', 'appreciates', 'comfort', 'quality', 'rolled', 'one', 'BlueLt', 'Blue', 'Handle']


In [38]:
from gensim.models import FastText

model = FastText(vector_size = 25, window = 5, min_count = 5, sentences = product_descriptions, epochs = 1)

In [39]:
model.save("./fast_text_product_descriptions_5000-25-5-5.model")

In [33]:
model.wv.most_similar("apple")

[('Apple', 0.9999829530715942),
 ('hits', 0.9999563097953796),
 ('replacement', 0.9999547004699707),
 ('coordinated', 0.9999493360519409),
 ('replenish', 0.9999470114707947),
 ('darts', 0.9999464750289917),
 ('fermented', 0.9999457597732544),
 ('gets', 0.9999454617500305),
 ('Decaffeinated', 0.9999440312385559),
 ('belts', 0.9999421238899231)]

In [34]:
model.wv.vector_size

25

In [35]:
from gensim.models import fasttext
fasttext.save_facebook_model(model, "./fast_text_product_descriptions_10000-25-5-5.bin")

In [36]:
model = fasttext.load_facebook_model("./fast_text_product_descriptions_10000-25-5-5.bin")

In [37]:
model.wv.most_similar("apple")

[('Apple', 0.9999829530715942),
 ('hits', 0.9999563097953796),
 ('replacement', 0.9999547004699707),
 ('coordinated', 0.9999493360519409),
 ('replenish', 0.9999470114707947),
 ('darts', 0.9999464750289917),
 ('fermented', 0.9999457597732544),
 ('gets', 0.9999454617500305),
 ('Decaffeinated', 0.9999440312385559),
 ('belts', 0.9999421238899231)]

The fastText model seems a bit too big to be stored on Github and has to be generated by the recommendations server if it has to be stored on a Volume in Railway.

I'm going to test out if just training a word2vec model on these still does well enough, and whether it is any smaller.

In [41]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = product_descriptions, vector_size = 25, window = 5, min_count = 5)
model.save("./word2vec_product_descriptions_10000-25-5-5.model")

In [42]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load("word2vec_product_descriptions_10000-25-5-5.model", mmap='r')

In [44]:
wv.wv.most_similar("apple")

[('Mix', 0.9916365742683411),
 ('paste', 0.9915204644203186),
 ('seeds', 0.9914876818656921),
 ('onions', 0.9904603958129883),
 ('potato', 0.990379810333252),
 ('Delicious', 0.9899361729621887),
 ('grilled', 0.9893227815628052),
 ('cardamom', 0.9890508651733398),
 ('grass', 0.9883642196655273),
 ('tangy', 0.9881302714347839)]